In [1]:
import pandas as pd
import os
from glob import glob

# Define input and output folders
input_folder = "Datasets/salary"
output_folder = "Datasets/Salaries"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Required columns in lowercase
required_columns = [
    "occ_code", "occ_title", "area_title",
    "naics_title", "a_median", "a_pct10", "a_pct90"
]

# Get all Excel files from the folder
excel_files = glob(os.path.join(input_folder, "*.xlsx"))

# Process each file
for file_path in excel_files:
    try:
        filename = os.path.basename(file_path)
        year = next((part for part in filename.split('_') if part.isdigit()), None)

        # Read the Excel file
        df = pd.read_excel(file_path)

        # Convert column names to lowercase
        df.columns = [col.lower() for col in df.columns]

        # Add year column
        df["year"] = year

        # Reorder and filter columns (now all lowercase)
        filtered_df = df[required_columns + ["year"]]

        # Write to CSV
        output_path = os.path.join(output_folder, f"filtered_occupation_data_{year}.csv")
        filtered_df.to_csv(output_path, index=False)

        # Uncomment for debug:
        print(f"✅ Processed and saved: {output_path}")

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")


✅ Processed and saved: Datasets/Salaries/filtered_occupation_data_2020.csv
✅ Processed and saved: Datasets/Salaries/filtered_occupation_data_2021.csv
✅ Processed and saved: Datasets/Salaries/filtered_occupation_data_2023.csv
✅ Processed and saved: Datasets/Salaries/filtered_occupation_data_2024.csv
✅ Processed and saved: Datasets/Salaries/filtered_occupation_data_2019.csv
✅ Processed and saved: Datasets/Salaries/filtered_occupation_data_2016.csv
✅ Processed and saved: Datasets/Salaries/filtered_occupation_data_2018.csv
✅ Processed and saved: Datasets/Salaries/filtered_occupation_data_2017.csv
✅ Processed and saved: Datasets/Salaries/filtered_occupation_data_2022.csv


In [5]:
import pandas as pd
import json
import csv
import os
from glob import glob

# Directory containing salary_data_2016.csv to salary_data_2024.csv
dataset_dir = "Datasets/"
salary_dir = f"{dataset_dir}Salaries/"
salary_files = sorted(glob(os.path.join(salary_dir, "filtered_occupation_data_*.csv")))

# Check if salary files exist
if not salary_files:
    raise FileNotFoundError("❌ No salary_data_*.csv files found in the current directory.")

print(f"✅ Found salary files: {[os.path.basename(f) for f in salary_files]}")

# Load all salary CSVs
salary_dfs = []
for file in salary_files:
    try:
        df = pd.read_csv(file)
        df["YEAR"] = int(file.split("_")[-1].split(".")[0])
        salary_dfs.append(df)
    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")

salary_df = pd.concat(salary_dfs, ignore_index=True)

# Load education and skills
education_df = pd.read_csv(os.path.join(dataset_dir, "education_data.csv"))
skills_df = pd.read_csv(os.path.join(dataset_dir, "skills_data.csv"))

# Ensure salary columns are numeric before division
salary_df["a_pct10"] = pd.to_numeric(salary_df["a_pct10"], errors="coerce")
salary_df["a_median"] = pd.to_numeric(salary_df["a_median"], errors="coerce")
salary_df["a_pct90"] = pd.to_numeric(salary_df["a_pct90"], errors="coerce")

# Convert annual to monthly
salary_df["M_PCT10"] = (salary_df["a_pct10"] / 12).round(2)
salary_df["M_MEDIAN"] = (salary_df["a_median"] / 12).round(2)
salary_df["M_PCT90"] = (salary_df["a_pct90"] / 12).round(2)

# Education mapping
edu_map = {
    "Less_than_hs": "LESS_THAN_HS",
    "hs_or_eq": "HIGH_SCHOOL",
    "Associate_degree": "ASSOCIATE",
    "Bachelor_degree": "BACHELOR",
    "Master_degree": "MASTERS",
    "Doctorate_degree": "DOCTORATE",
    "No_requirement": "NO_REQ",
    "Professional_degree": "PROFESSIONAL"
}

# Build records
records = []
grouped_salary = salary_df.groupby(["occ_code", "year"])

for (soc_code, year), group in grouped_salary:
    existing_record = next((r for r in records if r["soc_code"] == soc_code), None)

    if not existing_record:
        existing_record = {
            "soc_code": soc_code,
            "title": group.iloc[0]["occ_title"],
            "description": "",
            "salary": {},
            "education": {key: "" for key in edu_map.values()},
            "typicalSkills": []
        }
        records.append(existing_record)

    year_str = str(year)
    existing_record["salary"].setdefault(year_str, {})

    for _, row in group.iterrows():
        state = row["area_title"]
        industry = row["naics_title"]
        existing_record["salary"][year_str].setdefault(state, {})[industry] = {
            "A_MEDIAN": float(row["a_median"]),
            "M_PCT10": float(row["M_PCT10"]),
            "M_MEDIAN": float(row["M_MEDIAN"]),
            "M_PCT90": float(row["M_PCT90"])
        }

# Add education data
for record in records:
    soc_code = record["soc_code"]
    edu_rows = education_df[(education_df["SOC"] == soc_code) | (education_df["SOC"] == "00-0000")]
    for _, edu_row in edu_rows.iterrows():
        est_code = edu_row["ESTIMATECODE"]
        est_value = str(edu_row["ESTIMATE"])
        if est_code in edu_map:
            record["education"][edu_map[est_code]] = est_value

# Add skills data
for record in records:
    soc_code = record["soc_code"]
    skills_row = skills_df[skills_df["SOC_CODE"] == soc_code]
    if not skills_row.empty:
        raw_skills = skills_row.iloc[0]["TYPICAL_SKILLS"].replace("'", "\"")
        try:
            skills_list = json.loads(raw_skills)
            record["typicalSkills"] = sorted(set(skills_list))
        except json.JSONDecodeError:
            print(f"⚠️ Error decoding skills for SOC {soc_code}")
            record["typicalSkills"] = []

# Write to output CSV
output_file = "soc_compiled_output.csv"
with open(output_file, "w", newline='', encoding="utf-8") as csvfile:
    fieldnames = ["soc_code", "title", "description", "salary", "education", "typicalSkills"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for rec in records:
        writer.writerow({
            "soc_code": rec["soc_code"],
            "title": rec["title"],
            "description": rec["description"],
            "salary": json.dumps(rec["salary"]),
            "education": json.dumps(rec["education"]),
            "typicalSkills": json.dumps(rec["typicalSkills"])
        })

print(f"✅ Done! Output written to '{output_file}' with {len(records)} SOC records.")


✅ Found salary files: ['filtered_occupation_data_2016.csv', 'filtered_occupation_data_2017.csv', 'filtered_occupation_data_2018.csv', 'filtered_occupation_data_2019.csv', 'filtered_occupation_data_2020.csv', 'filtered_occupation_data_2021.csv', 'filtered_occupation_data_2022.csv', 'filtered_occupation_data_2023.csv', 'filtered_occupation_data_2024.csv']
✅ Done! Output written to 'soc_compiled_output.csv' with 1550 SOC records.
